<span style = "font-family: Verdana; font-size: 20px">

#### **Preprocessing**
</span>

<span style = "font-family: Verdana; font-size: 20px">

##### **Các bước thực hiện và ý tưởng**

Dựa trên đặc tính của bộ dữ liệu và mục tiêu dự án, các bước tiền xử lý dữ liệu dự kiến sẽ được thực hiện như sau:
- Xử lý các vấn đề tìm được trong cả 3 file dữ liệu
- Kết hợp dữ liệu từ `pokemon.csv` và `pokemonfullstats.csv` để tạo thành một bảng dữ liệu đầy đủ về đặc tính của các Pokemon
- Cuối cùng, kết hợp dữ liệu từ bảng đặc tính Pokemon với bảng trận đấu `combats.csv` để tạo thành bảng dữ liệu cuối cùng dùng để huấn luyện mô hình dự đoán kết quả trận đấu giữa hai Pokemon
</span>

In [876]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [877]:
pokemon = pd.read_csv('data/pokemon.csv')
combats = pd.read_csv('data/combats.csv')
pokemonfullstats = pd.read_csv('data/pokemonfullstats.csv')

In [878]:
pokemon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   #           800 non-null    int64 
 1   Name        799 non-null    object
 2   Type 1      800 non-null    object
 3   Type 2      414 non-null    object
 4   HP          800 non-null    int64 
 5   Attack      800 non-null    int64 
 6   Defense     800 non-null    int64 
 7   Sp. Atk     800 non-null    int64 
 8   Sp. Def     800 non-null    int64 
 9   Speed       800 non-null    int64 
 10  Generation  800 non-null    int64 
 11  Legendary   800 non-null    bool  
dtypes: bool(1), int64(8), object(3)
memory usage: 69.7+ KB


In [879]:
pokemonfullstats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1179 entries, 0 to 1178
Data columns (total 47 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   DexNumber           1179 non-null   int64  
 1   Name                1179 non-null   object 
 2   Type                1179 non-null   object 
 3   Abilities           1179 non-null   object 
 4   HiddenAbility       1179 non-null   object 
 5   Generation          1179 non-null   object 
 6   Hp                  1179 non-null   int64  
 7   Attack              1179 non-null   int64  
 8   Defense             1179 non-null   int64  
 9   SpecialAttack       1179 non-null   int64  
 10  SpecialDefense      1179 non-null   int64  
 11  Speed               1179 non-null   int64  
 12  TotalStats          1179 non-null   int64  
 13  Weight              1179 non-null   float64
 14  Height              1179 non-null   float64
 15  GenderProbM         1179 non-null   object 
 16  Catego

In [880]:
combats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   First_pokemon   50000 non-null  int64
 1   Second_pokemon  50000 non-null  int64
 2   Winner          50000 non-null  int64
dtypes: int64(3)
memory usage: 1.1 MB


<span style = "font-family: Verdana; font-size: 20px">

##### **1. Missing Values**
</span>

In [881]:
na_pokemon = pokemon.isna().sum()
na_pokemonfs = pokemonfullstats.isna().sum()
na_combats = combats.isna().sum()
print(f'Missing value in pokemon: {sum(na_pokemon[na_pokemon > 0])}')
print(f'Missing value in pokemonfullstats: {sum(na_pokemonfs[na_pokemonfs > 0])}')
print(f'Missing value in combats: {sum(na_combats[na_combats > 0])}')

Missing value in pokemon: 387
Missing value in pokemonfullstats: 0
Missing value in combats: 0


<span style = "font-family: Verdana; font-size: 20px">

- Dựa vào hàm .info() ở trên ta thấy `pokemon.csv` có 1 missing value ở cột `Name` và rất nhiều missing values ở cột `Type 2` do nhiều Pokemon chỉ có một thuộc tính chính
</span>

<span style = "font-family: Verdana; font-size: 20px">

Ở cột Type 2, missing value có thể được giải quyết đơn giản bằng cách thêm giá trị 'None' để biểu thị cho việc Pokemon đó chỉ có một thuộc tính chính
</span>

In [882]:
pokemon.fillna({'Type 2': 'None'}, inplace=True)

<span style = "font-family: Verdana; font-size: 20px">

Còn ở cột `Name`, vì tên của các Pokemon thường sẽ được xếp theo thứ tự tiến hoá nên hãy xem qua các dòng xung quanh của missing value này
</span>

In [883]:
missing_point = pokemon.index[pokemon['Name'].isna()][0]
pokemon.loc[missing_point - 3:missing_point + 3, ['Name', 'Type 1', 'Type 2']]

,Name,Type 1,Type 2
59,Psyduck,Water,None
60,Golduck,Water,None
61,Mankey,Fighting,None
62,NaN,Fighting,None
63,Growlithe,Fire,None
64,Arcanine,Fire,None
65,Poliwag,Water,None


<span style = "font-family: Verdana; font-size: 20px">

Như vậy giá trị bị thiếu nằm giữa hai Pokemon có tên 'Mankey' và 'Growlithe', có thể suy luận giá trị từ các web tổng hợp thông tin pokemon, hoặc xem từ `pokemonfullstats.csv` vì dataset này không có giá trị thiếu trên cột `Name`

In [884]:
target_point = pokemonfullstats.index[pokemonfullstats['Name'] == 'Mankey'][0]
pokemonfullstats.loc[target_point - 2:target_point + 4, ['Name', 'Type']]

,Name,Type
65,Psyduck,['Water']
66,Golduck,['Water']
67,Mankey,['Fighting']
68,Primeape,['Fighting']
69,Growlithe,['Fire']
70,Hisuian Growlithe,"['Fire', 'Rock']"
71,Arcanine,['Fire']


<span style = "font-family: Verdana; font-size: 20px">

Có thể thấy giá trị bị thiếu chính là 'Primeape'
</span>

In [885]:
pokemon.loc[62, "Name"] = 'Primeape'

<span style = "font-family: Verdana; font-size: 20px">

Kiểm tra lại để chắc chắc không còn missing values nào trong toàn bộ dataset
</span>

In [886]:
na_pokemon = pokemon.isna().sum()
na_pokemonfs = pokemonfullstats.isna().sum()
na_combats = combats.isna().sum()
print(f'Missing value in pokemon: {sum(na_pokemon[na_pokemon > 0])}')
print(f'Missing value in pokemonfullstats: {sum(na_pokemonfs[na_pokemonfs > 0])}')
print(f'Missing value in combats: {sum(na_combats[na_combats > 0])}')

Missing value in pokemon: 0
Missing value in pokemonfullstats: 0
Missing value in combats: 0


<span style = "font-family: Verdana; font-size: 20px">

##### **2. Duplicating Values**
</span>

In [887]:
dup_pokemon = pokemon.duplicated().sum()
dup_pokemonfs = pokemonfullstats.duplicated().sum()
dup_combats = combats.duplicated().sum()
print(f'Duplicated value in pokemon: {sum(dup_pokemon[dup_pokemon > 0])}')
print(f'Duplicated value in pokemonfullstats: {sum(dup_pokemonfs[dup_pokemonfs > 0])}')
print(f'Duplicated value in combats: {sum(dup_combats[dup_combats > 0])}')

Duplicated value in pokemon: 0
Duplicated value in pokemonfullstats: 0
Duplicated value in combats: 1952


<span style = "font-family: Verdana; font-size: 20px">

Với dữ liệu lặp, cách xử lý đơn giản nhất là loại bỏ các dòng bị lặp đi
</span>

In [888]:
dup_combats_indices = combats[combats.duplicated()].index
combats = combats.drop(dup_combats_indices).reset_index(drop=True)

<span style = "font-family: Verdana; font-size: 20px">

Kiểm tra lại để chắc chắc không còn dữ liệu lặp nào trong toàn bộ dataset
</span>

In [889]:
dup_pokemon = pokemon.duplicated().sum()
dup_pokemonfs = pokemonfullstats.duplicated().sum()
dup_combats = combats.duplicated().sum()
print(f'Duplicated value in pokemon: {sum(dup_pokemon[dup_pokemon > 0])}')
print(f'Duplicated value in pokemonfullstats: {sum(dup_pokemonfs[dup_pokemonfs > 0])}')
print(f'Duplicated value in combats: {sum(dup_combats[dup_combats > 0])}')

Duplicated value in pokemon: 0
Duplicated value in pokemonfullstats: 0
Duplicated value in combats: 0


<span style = "font-family: Verdana; font-size: 20px">

##### **3. Các vấn đề đặc thù của bộ dữ liệu**
</span>

In [890]:
pd.options.display.max_columns = None

In [891]:
pokemon.head(10)

,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False
3,4,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,1,False
4,5,Charmander,Fire,None,39,52,43,60,50,65,1,False
5,6,Charmeleon,Fire,None,58,64,58,80,65,80,1,False
6,7,Charizard,Fire,Flying,78,84,78,109,85,100,1,False
7,8,Mega Charizard X,Fire,Dragon,78,130,111,130,85,100,1,False
8,9,Mega Charizard Y,Fire,Flying,78,104,78,159,115,100,1,False
9,10,Squirtle,Water,None,44,48,65,50,64,43,1,False


In [892]:
pokemonfullstats.head(10)

,DexNumber,Name,Type,Abilities,HiddenAbility,Generation,Hp,Attack,Defense,SpecialAttack,SpecialDefense,Speed,TotalStats,Weight,Height,GenderProbM,Category,CatchRate,EggCycles,EggGroup,LevelingRate,BaseFriendship,IsLegendary,IsMythical,IsUltraBeast,HasMega,EvoStage,TotalEvoStages,PreevoName,DamageFromNormal,DamageFromFighting,DamageFromFlying,DamageFromPoison,DamageFromGround,DamageFromRock,DamageFromBug,DamageFromGhost,DamageFromSteel,DamageFromFire,DamageFromWater,DamageFromGrass,DamageFromElectric,DamageFromPsychic,DamageFromIce,DamageFromDragon,DamageFromDark,DamageFromFairy
0,1,Bulbasaur,"['Grass', 'Poison']",['Overgrow'],['Chlorophyll'],I,45,49,49,65,65,45,318,6.9,0.7,0.875,Seed Pokémon,45,20,['Monster' 'Grass'],Medium Slow\n,70,0,0,0,0,1,3,No Preevolution,1.0,0.5,2.0,1.0,1.0,1.0,1.00,1.0,1.0,2.0,0.5,0.25,0.5,2.0,2.0,1.0,1.0,0.5
1,2,Ivysaur,"['Grass', 'Poison']",['Overgrow'],['Chlorophyll'],I,60,62,63,80,80,60,405,13.0,1.0,0.875,Seed Pokémon,45,20,['Monster' 'Grass'],Medium Slow\n,70,0,0,0,0,2,3,Bulbasaur,1.0,0.5,2.0,1.0,1.0,1.0,1.00,1.0,1.0,2.0,0.5,0.25,0.5,2.0,2.0,1.0,1.0,0.5
2,3,Venusaur,"['Grass', 'Poison']",['Overgrow'],['Chlorophyll'],I,80,82,83,100,100,80,625,100.0,2.0,0.875,Seed Pokémon,45,20,['Monster' 'Grass'],Medium Slow\n,70,0,0,0,1,3,3,Ivysaur,1.0,0.5,2.0,1.0,1.0,1.0,1.00,1.0,1.0,2.0,0.5,0.25,0.5,2.0,2.0,1.0,1.0,0.5
3,4,Charmander,['Fire'],['Blaze'],['Solar Power'],I,39,52,43,60,50,65,309,8.5,0.6,0.875,Lizard Pokémon,45,20,['Monster' 'Dragon'],Medium Slow\n,70,0,0,0,0,1,3,No Preevolution,1.0,1.0,1.0,1.0,2.0,2.0,0.50,1.0,0.5,0.5,2.0,0.50,1.0,1.0,0.5,1.0,1.0,0.5
4,5,Charmeleon,['Fire'],['Blaze'],['Solar Power'],I,58,64,58,80,65,80,405,19.0,1.1,0.875,Flame Pokémon,45,20,['Monster' 'Dragon'],Medium Slow\n,70,0,0,0,0,2,3,Charmander,1.0,1.0,1.0,1.0,2.0,2.0,0.50,1.0,0.5,0.5,2.0,0.50,1.0,1.0,0.5,1.0,1.0,0.5
5,6,Charizard,"['Fire', 'Flying']",['Blaze'],['Solar Power'],I,78,84,78,109,85,100,634,90.5,1.7,0.875,Flame Pokémon,45,20,['Monster' 'Dragon'],Medium Slow\n,70,0,0,0,1,3,3,Charmeleon,1.0,0.5,1.0,1.0,0.0,4.0,0.25,1.0,0.5,0.5,2.0,0.25,2.0,1.0,1.0,1.0,1.0,0.5
6,7,Squirtle,['Water'],['Torrent'],['Rain Dish'],I,44,48,65,50,64,43,314,9.0,0.5,0.875,Tiny Turtle Pokémon,45,20,['Monster' 'Water 1'],Medium Slow\n,70,0,0,0,0,1,3,No Preevolution,1.0,1.0,1.0,1.0,1.0,1.0,1.00,1.0,0.5,0.5,0.5,2.00,2.0,1.0,0.5,1.0,1.0,1.0
7,8,Wartortle,['Water'],['Torrent'],['Rain Dish'],I,59,63,80,65,80,58,405,22.5,1.0,0.875,Turtle Pokémon,45,20,['Monster' 'Water 1'],Medium Slow\n,70,0,0,0,0,2,3,Squirtle,1.0,1.0,1.0,1.0,1.0,1.0,1.00,1.0,0.5,0.5,0.5,2.00,2.0,1.0,0.5,1.0,1.0,1.0
8,9,Blastoise,['Water'],['Torrent'],['Rain Dish'],I,79,83,100,85,105,78,630,85.5,1.6,0.875,Shellfish Pokémon,45,20,['Monster' 'Water 1'],Medium Slow\n,70,0,0,0,1,3,3,Wartortle,1.0,1.0,1.0,1.0,1.0,1.0,1.00,1.0,0.5,0.5,0.5,2.00,2.0,1.0,0.5,1.0,1.0,1.0
9,10,Caterpie,['Bug'],['Shield Dust'],['Run Away'],I,45,30,35,20,20,45,195,2.9,0.3,0.5,Worm Pokémon,255,15,['Bug'],Medium Fast\n,70,0,0,0,0,1,3,No Preevolution,1.0,0.5,2.0,1.0,0.5,2.0,1.00,1.0,1.0,2.0,1.0,0.50,1.0,1.0,1.0,1.0,1.0,1.0


<span style = "font-family: Verdana; font-size: 20px">

Để trực quan hoá một cách hiệu quả bộ dữ liệu về Pokemon, trước hết cần kết hợp thông tin từ cả hai bộ dữ liệu `pokemon.csv` và `pokemonfullstats.csv` để có được bảng dữ liệu đầy đủ về đặc tính của các Pokemon
Nhưng có một số vấn đề cần phải giải quyết trước khi thực hiện việc này:
- Dễ thấy cách đánh số ID của các Pokemon trong hai bộ dữ liệu không đồng nhất, ví dụ như trong `pokemon.csv`, Charmander có ID ở cột `#` là 5, nhưng trong `pokemonfullstats.csv`, lại có DexNumber là 4, và sự chênh lệch này sẽ lặp lại đến cuối bộ dữ liệu
</span>

In [893]:
pd.concat([pokemon[['#', 'Name']].head(10),pokemonfullstats[['DexNumber', 'Name']].head(10)], axis=1)

,#,Name,DexNumber,Name
0,1,Bulbasaur,1,Bulbasaur
1,2,Ivysaur,2,Ivysaur
2,3,Venusaur,3,Venusaur
3,4,Mega Venusaur,4,Charmander
4,5,Charmander,5,Charmeleon
5,6,Charmeleon,6,Charizard
6,7,Charizard,7,Squirtle
7,8,Mega Charizard X,8,Wartortle
8,9,Mega Charizard Y,9,Blastoise
9,10,Squirtle,10,Caterpie


<span style = "font-family: Verdana; font-size: 20px">

Nguyên nhân gây ra sự chênh lệch này là do trong `pokemonfullstats.csv`, dữ liệu được lấy từ [serebii.net](https://serebii.net/pokemon) và [bulbapedia](https://bulbapedia.bulbagarden.net/), và theo quy ước của các trang web này, các Pokemon tiến hoá ở bậc Mega sẽ nằm cùng một trang với Pokemon gốc của nó và có cùng số Pokedex, rất có thể thay đổi này được thực hiện sau khi bộ dữ liệu `pokemon.csv` được thu thập dẫn đến sự chênh lệch này

Tuy nhiên bộ dữ liệu `pokemon.csv` có cùng nguồn gốc với `combats.csv`, nên để đảm bảo tính nhất quán của dữ liệu, chúng em vẫn sẽ sử dụng cách đánh số ID từ `pokemon.csv` làm chuẩn, và `merge` hai bảng dữ liệu dựa trên cột `Name` thay vì cột `#` và `DexNumber` như thông thường, điều này đồng nghĩa cột `DexNumber` trong `pokemonfullstats.csv` sẽ không còn giá trị sử dụng
</span>

In [894]:
pokemonfullstats.drop(columns=['DexNumber'], inplace=True)

<span style = "font-family: Verdana; font-size: 20px">

- Ở cột `LevelingRate` trong `pokemonfullstats.csv`, có một ký tự xuống dòng thừa ở cuối mỗi giá trị, cần loại bỏ ký tự này để tránh lỗi khi xử lý dữ liệu sau này
</span>

In [895]:
pokemonfullstats['LevelingRate'] = pokemonfullstats['LevelingRate'].str.rstrip('\n')

<span style = "font-family: Verdana; font-size: 20px">

- Tiếp đó, trong cả 2 bộ dữ liệu cũng có một số cột có nội dung tương tự nhau, ví dụ như các chỉ số chiến đấu (HP, Attack, Defense, Special Attack, Special Defense, Speed), để tránh việc dư thừa thông tin khi kết hợp hai bảng dữ liệu, chúng em sẽ giữ lại các cột từ `pokemonfullstats.csv` vì bộ dữ liệu này có nhiều thông tin chi tiết hơn về các đặc điểm của Pokemon và loại bỏ các cột tương ứng từ `pokemon.csv`
</span>

In [896]:
pokemon.drop(columns=['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed', 'Generation', 'Legendary'], inplace=True)

In [897]:
merged_pokemon = pd.merge(pokemon, pokemonfullstats, left_on='Name', right_on='Name', how='outer')
merged_pokemon = merged_pokemon.dropna(subset=['#'])
merged_pokemon.sort_values(by=['#'], inplace=True, ignore_index=True)
merged_pokemon.rename(columns={'#': 'ID'}, inplace=True)

In [898]:
merged_pokemon.head(10)

,ID,Name,Type 1,Type 2,Type,Abilities,HiddenAbility,Generation,Hp,Attack,Defense,SpecialAttack,SpecialDefense,Speed,TotalStats,Weight,Height,GenderProbM,Category,CatchRate,EggCycles,EggGroup,LevelingRate,BaseFriendship,IsLegendary,IsMythical,IsUltraBeast,HasMega,EvoStage,TotalEvoStages,PreevoName,DamageFromNormal,DamageFromFighting,DamageFromFlying,DamageFromPoison,DamageFromGround,DamageFromRock,DamageFromBug,DamageFromGhost,DamageFromSteel,DamageFromFire,DamageFromWater,DamageFromGrass,DamageFromElectric,DamageFromPsychic,DamageFromIce,DamageFromDragon,DamageFromDark,DamageFromFairy
0,1.0,Bulbasaur,Grass,Poison,"['Grass', 'Poison']",['Overgrow'],['Chlorophyll'],I,45.0,49.0,49.0,65.0,65.0,45.0,318.0,6.9,0.7,0.875,Seed Pokémon,45.0,20.0,['Monster' 'Grass'],Medium Slow,70.0,0.0,0.0,0.0,0.0,1.0,3.0,No Preevolution,1.0,0.5,2.0,1.0,1.0,1.0,1.00,1.0,1.0,2.0,0.5,0.25,0.5,2.0,2.0,1.0,1.0,0.5
1,2.0,Ivysaur,Grass,Poison,"['Grass', 'Poison']",['Overgrow'],['Chlorophyll'],I,60.0,62.0,63.0,80.0,80.0,60.0,405.0,13.0,1.0,0.875,Seed Pokémon,45.0,20.0,['Monster' 'Grass'],Medium Slow,70.0,0.0,0.0,0.0,0.0,2.0,3.0,Bulbasaur,1.0,0.5,2.0,1.0,1.0,1.0,1.00,1.0,1.0,2.0,0.5,0.25,0.5,2.0,2.0,1.0,1.0,0.5
2,3.0,Venusaur,Grass,Poison,"['Grass', 'Poison']",['Overgrow'],['Chlorophyll'],I,80.0,82.0,83.0,100.0,100.0,80.0,625.0,100.0,2.0,0.875,Seed Pokémon,45.0,20.0,['Monster' 'Grass'],Medium Slow,70.0,0.0,0.0,0.0,1.0,3.0,3.0,Ivysaur,1.0,0.5,2.0,1.0,1.0,1.0,1.00,1.0,1.0,2.0,0.5,0.25,0.5,2.0,2.0,1.0,1.0,0.5
3,4.0,Mega Venusaur,Grass,Poison,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5.0,Charmander,Fire,None,['Fire'],['Blaze'],['Solar Power'],I,39.0,52.0,43.0,60.0,50.0,65.0,309.0,8.5,0.6,0.875,Lizard Pokémon,45.0,20.0,['Monster' 'Dragon'],Medium Slow,70.0,0.0,0.0,0.0,0.0,1.0,3.0,No Preevolution,1.0,1.0,1.0,1.0,2.0,2.0,0.50,1.0,0.5,0.5,2.0,0.50,1.0,1.0,0.5,1.0,1.0,0.5
5,6.0,Charmeleon,Fire,None,['Fire'],['Blaze'],['Solar Power'],I,58.0,64.0,58.0,80.0,65.0,80.0,405.0,19.0,1.1,0.875,Flame Pokémon,45.0,20.0,['Monster' 'Dragon'],Medium Slow,70.0,0.0,0.0,0.0,0.0,2.0,3.0,Charmander,1.0,1.0,1.0,1.0,2.0,2.0,0.50,1.0,0.5,0.5,2.0,0.50,1.0,1.0,0.5,1.0,1.0,0.5
6,7.0,Charizard,Fire,Flying,"['Fire', 'Flying']",['Blaze'],['Solar Power'],I,78.0,84.0,78.0,109.0,85.0,100.0,634.0,90.5,1.7,0.875,Flame Pokémon,45.0,20.0,['Monster' 'Dragon'],Medium Slow,70.0,0.0,0.0,0.0,1.0,3.0,3.0,Charmeleon,1.0,0.5,1.0,1.0,0.0,4.0,0.25,1.0,0.5,0.5,2.0,0.25,2.0,1.0,1.0,1.0,1.0,0.5
7,8.0,Mega Charizard X,Fire,Dragon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,9.0,Mega Charizard Y,Fire,Flying,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10.0,Squirtle,Water,None,['Water'],['Torrent'],['Rain Dish'],I,44.0,48.0,65.0,50.0,64.0,43.0,314.0,9.0,0.5,0.875,Tiny Turtle Pokémon,45.0,20.0,['Monster' 'Water 1'],Medium Slow,70.0,0.0,0.0,0.0,0.0,1.0,3.0,No Preevolution,1.0,1.0,1.0,1.0,1.0,1.0,1.00,1.0,0.5,0.5,0.5,2.00,2.0,1.0,0.5,1.0,1.0,1.0


<span style = "font-family: Verdana; font-size: 20px">

##### **3.1. Missing Values**
</span>

In [899]:
merged_pokemon.isna().sum()

ID                    0
Name                  0
Type 1                0
Type 2                0
Type                 81
                     ..
DamageFromPsychic    81
DamageFromIce        81
DamageFromDragon     81
DamageFromDark       81
DamageFromFairy      81
Length: 49, dtype: int64

<span style = "font-family: Verdana; font-size: 20px">

Sau khi kết hợp hai bảng dữ liệu, sẽ tạo ra kha khá missing values, nhưng điều này có thể dự đoán trước vì như đã đề cập ở trên, dữ liệu về các Pokemon sau khi tiến hoá bậc Mega không được thu thập, nhưng số lượng này không quá lớn, cho nên việc tra cứu và bổ sung là khả thi

Thêm vào đó, cột `Type 1` và `Type 2` không có missing values, chứng tỏ không có Pokemon nào nằm ngoài bộ dữ liệu `pokemon.csv` vô tình lọt vào, do đó cột `Type` cũng có thể được lược bỏ
</span>

In [900]:
merged_pokemon = merged_pokemon.drop(columns=['Type'])

<span style = "font-family: Verdana; font-size: 20px">

Để chắc chắn tất cả các Pokemon bị thiếu thông tin đều ở dạng tiến hoá bậc Mega
</span>

In [901]:
pd.options.display.max_rows = None
missing_pokemon = merged_pokemon[merged_pokemon.isna().any(axis=1)]['Name']
print(missing_pokemon)

3               Mega Venusaur
7            Mega Charizard X
8            Mega Charizard Y
12             Mega Blastoise
19              Mega Beedrill
23               Mega Pidgeot
71              Mega Alakazam
87               Mega Slowbro
102               Mega Gengar
124           Mega Kangaskhan
137               Mega Pinsir
141             Mega Gyarados
154           Mega Aerodactyl
163             Mega Mewtwo X
164             Mega Mewtwo Y
196             Mega Ampharos
216                     Unown
224              Mega Steelix
229               Mega Scizor
232            Mega Heracross
248             Mega Houndoom
268            Mega Tyranitar
270                     Ho-oh
275             Mega Sceptile
279             Mega Blaziken
283             Mega Swampert
306            Mega Gardevoir
327              Mega Sableye
329               Mega Mawile
333               Mega Aggron
336             Mega Medicham
339            Mega Manectric
349             Mega Sharpedo
354       

<span style = "font-family: Verdana; font-size: 20px">

Có khá nhiều Pokemon bình thường bị thiếu thông tin, chứ không chỉ có các dạng tiến hoá bậc Mega như đã dự đoán

In [902]:
non_mega = missing_pokemon[~missing_pokemon.str.startswith('Mega')]
print(non_mega)
pd.options.display.max_rows = 10

216                     Unown
270                     Ho-oh
383                  Castform
422             Primal Kyogre
424            Primal Groudon
429        DeoxysAttack Forme
457                     Burmy
468                   Cherrim
469                   Shellos
470                 Gastrodon
610                  Basculin
646                  Deerling
647                  Sawsbuck
711       Kyurem Black Kyurem
712       Kyurem White Kyurem
713     Keldeo Ordinary Forme
714     Keldeo Resolute Forme
717                  Genesect
734                  Vivillon
737                   Flabébé
738                   Floette
739                   Florges
744                   Furfrou
780    Pumpkaboo Average Size
781      Pumpkaboo Small Size
782      Pumpkaboo Large Size
783      Pumpkaboo Super Size
784    Gourgeist Average Size
785      Gourgeist Small Size
786      Gourgeist Large Size
787      Gourgeist Super Size
792                   Xerneas
794        Zygarde Half Forme
Name: Name

<span style = "font-family: Verdana; font-size: 20px">

Đầu tiên hãy kiểm tra xem các Pokemon bị thiếu thông tin này có trong `pokemonfullstats.csv` dưới cách đặt tên khác hay không

Tuy nhiên cũng có một số Pokemon có khả năng là bị thiếu thông tin do tên được điền sai, điển hình như `Primal Kyogre`, `Primal Groudon`, `DeoxysAttack Forme`, `Kyurem Black Kyurem`, `Kyurem White Kyurem`, `Keldeo Ordinary Forme`, `Keldeo Resolute Forme` và `Zygarde Half Forme`, những ví dụ này sẽ được xử lý riêng sau

Một số Pokemon khác có thể do cách phân loại như `Pumpkaboo` và `Gourgeist` cũng sẽ được kiểm tra và xử lý sau
</span>

In [903]:
non_mega_unique = non_mega.unique()
exclude_names = ['Primal Kyogre', 'Primal Groudon', 'DeoxysAttack Forme', 'Kyurem Black Kyurem', 'Kyurem White Kyurem', 'Keldeo Ordinary Forme', 'Keldeo Resolute Forme', 'Zygarde Half Forme', 'Pumpkaboo Average Size', 'Pumpkaboo Small Size', 'Pumpkaboo Large Size', 'Pumpkaboo Super Size', 'Gourgeist Average Size', 'Gourgeist Small Size', 'Gourgeist Large Size', 'Gourgeist Super Size']
non_mega_unique = np.append(non_mega_unique, ['Kyogre', 'Groudon', 'Deoxys', 'Kyurem', 'Keldeo', 'Zygarde', 'Pumpkaboo', 'Gourgeist'])
non_mega_filtered = [n for n in non_mega_unique if n not in exclude_names]
non_mega_filtered
for name in non_mega_filtered:
    info_in_pfs = pokemonfullstats[pokemonfullstats['Name'].str.contains(name, case=False, na=False)]
    if not info_in_pfs.empty:
        print(f"\n--- {name} ---")
        print(info_in_pfs[['Name', 'Type']])
    else:
        print(f"\n--- {name} --- NOT FOUND")


--- Unown ---
               Name         Type
237  Unown One Form  ['Psychic']

--- Ho-oh ---
      Name                Type
289  Ho-Oh  ['Fire', 'Flying']

--- Castform ---
                    Name        Type
392      Castform Normal  ['Normal']
393  Castform Sunny Form    ['Fire']
394  Castform Rainy Form   ['Water']
395  Castform Snowy Form     ['Ice']

--- Burmy ---
                  Name     Type
459  Burmy Plant Cloak  ['Bug']
460  Burmy Sandy Cloak  ['Bug']
461  Burmy Trash Cloak  ['Bug']

--- Cherrim ---
                      Name       Type
472  Cherrim Overcast Form  ['Grass']
473  Cherrim Sunshine Form  ['Grass']

--- Shellos ---
                 Name       Type
474  Shellos West Sea  ['Water']
475  Shellos East Sea  ['Water']

--- Gastrodon ---
                   Name                 Type
476  Gastrodon West Sea  ['Water', 'Ground']
477  Gastrodon East Sea  ['Water', 'Ground']

--- Basculin ---
                            Name       Type
615    Basculin Red-Striped Form 

<span style = "font-family: Verdana; font-size: 20px">

Vậy là mọi Pokemon bị thiếu thông tin ở trên đều là do tên của chúng không khớp với tên trong `pokemonfullstats.csv`, do đó việc bổ sung thông tin cho các Pokemon này sẽ được thực hiện bằng cách tham khảo nguồn và thêm lại từ `pokemonfullstats.csv`

In [904]:

def replace_data(pokemon_name, pfs_name):
    data = pokemonfullstats[pokemonfullstats['Name'] == pfs_name].iloc[0]
    index = merged_pokemon[merged_pokemon['Name'] == pokemon_name].index[0]
    for col in merged_pokemon.columns:
        if col not in ['ID', 'Name', 'Type 1', 'Type 2']:
            merged_pokemon.loc[index, col] = data[col]

# Unown
replace_data('Unown', 'Unown One Form')
        
# Ho-Oh
replace_data('Ho-oh', 'Ho-Oh')

# Castform, không chú thích nên mặc định sẽ để dạng Normal
replace_data('Castform', 'Castform Normal')

# Burmy có chỉ số gốc giống nhau cho tất cả các dạng
replace_data('Burmy', 'Burmy Plant Cloak')

# Cherrim có chỉ số gốc giống nhau cho tất cả các dạng
replace_data('Cherrim', 'Cherrim Overcast Form')

# Shellos có chỉ số gốc giống nhau cho tất cả các dạng
replace_data('Shellos', 'Shellos West Sea')

# Gastrodon là dạng tiến hoá của Shellos, tương tự
replace_data('Gastrodon', 'Gastrodon West Sea')

# Basculin có chỉ số gốc giống nhau cho tất cả các dạng
replace_data('Basculin', 'Basculin Red-Striped Form')

# Deerling có chỉ số gốc giống nhau cho tất cả các dạng
replace_data('Deerling', 'Deerling Spring Form')

# Sawsbuck là dạng tiến hoá của Deerling, tương tự
replace_data('Sawsbuck', 'Sawsbuck Spring Form')

# Genesect, không chú thích nên mặc định sẽ để dạng No Drive
replace_data('Genesect', 'No Drive Genesect')

# Vivillon
replace_data('Vivillon', 'Vivillon Meadow Pattern')

# Flabébé, các dạng chỉ thay đổi màu sắc, chỉ số gốc giống nhau
replace_data('Flabébé', 'Flabébé Red Flower')

# Floette là dạng tiến hoá của Flabébé, tương tự
replace_data('Floette', 'Floette Red Flower')

# Florges là dạng tiến hoá của Floette, tương tự
replace_data('Florges', 'Florges Red Flower')

# Furfrou
replace_data('Furfrou', 'Furfrou Natural Form')

# Xerneas, dạng Active chỉ thay đổi màu sừng, chỉ số gốc giống nhau
replace_data('Xerneas', 'Xerneas Neutral Mode')

# DeoxysAttack Forme, lỗi chính tả
replace_data('DeoxysAttack Forme', 'Deoxys Attack Forme')

# Kyurem Black Kyurem, Kyurem White Kyurem, cách ghi tên khác nhau
replace_data('Kyurem Black Kyurem', 'Black Kyurem')
replace_data('Kyurem White Kyurem', 'White Kyurem')

# Keldeo Ordinary Forme, Keldeo Resolute Forme, lỗi chính tả
replace_data('Keldeo Ordinary Forme', 'Keldeo Ordinary Form')
replace_data('Keldeo Resolute Forme', 'Keldeo Resolute Form')

# Zygarde Half Forme, cách ghi tên khác nhau
replace_data('Zygarde Half Forme', 'Zygarde 50% Forme')

<span style = "font-family: Verdana; font-size: 20px">

Cơ bản mọi Pokemon bị thiếu thông tin đều đã được bổ sung từ `pokemonfullstats.csv` nếu khả thi, chỉ còn một số ít Pokemon đặc biệt cần xử lý riêng như đã đề cập ở trên và các Pokemon tiến hoá bậc Mega, những Pokemon này sẽ được bổ sung thủ công từ thông tin từ các web đã được đề cập

In [905]:
pd.options.display.max_rows = None
missing_pokemon = merged_pokemon[merged_pokemon.isna().any(axis=1)]['Name']
print(missing_pokemon)
pd.options.display.max_rows = 10

3               Mega Venusaur
7            Mega Charizard X
8            Mega Charizard Y
12             Mega Blastoise
19              Mega Beedrill
23               Mega Pidgeot
71              Mega Alakazam
87               Mega Slowbro
102               Mega Gengar
124           Mega Kangaskhan
137               Mega Pinsir
141             Mega Gyarados
154           Mega Aerodactyl
163             Mega Mewtwo X
164             Mega Mewtwo Y
196             Mega Ampharos
224              Mega Steelix
229               Mega Scizor
232            Mega Heracross
248             Mega Houndoom
268            Mega Tyranitar
275             Mega Sceptile
279             Mega Blaziken
283             Mega Swampert
306            Mega Gardevoir
327              Mega Sableye
329               Mega Mawile
333               Mega Aggron
336             Mega Medicham
339            Mega Manectric
349             Mega Sharpedo
354             Mega Camerupt
366              Mega Altaria
387       

In [906]:
def add_data_mega_primal(pokemon_name, Abilities: str, HiddenAbility: str, Generation: str,
             HP: float, Attack: float, Defense: float, SpecialAttack: float,
             SpecialDefense: float, Speed: float, TotalStats: float,
             Weight: float, Height: float, GenderProbM: str, Category: str,
             CatchRate: float, EggCycles: float, EggGroups: list,
             LevelingRate: str, BaseFriendship: float, IsLegendary: float, IsMythical: float,
             EvoStage: float, TotalEvoStages: float,
             PreevoName: str, DamageFromNormal: float, DamageFromFighting: float,
             DamageFromFlying: float, DamageFromPoison: float, DamageFromGround: float,
             DamageFromRock: float, DamageFromBug: float, DamageFromGhost: float,
             DamageFromSteel: float, DamageFromFire: float, DamageFromWater: float,
             DamageFromGrass: float, DamageFromElectric: float, DamageFromPsychic: float,
             DamageFromIce: float, DamageFromDragon: float, DamageFromDark: float,
             DamageFromFairy: float, HasMega: float = 1.0, IsUltraBeast: float = 0.0
             ):
    index = merged_pokemon[merged_pokemon['Name'] == pokemon_name].index[0]
    merged_pokemon.loc[index, 'Abilities'] = Abilities
    merged_pokemon.loc[index, 'HiddenAbility'] = HiddenAbility
    merged_pokemon.loc[index, 'Generation'] = Generation
    merged_pokemon.loc[index, 'HP'] = HP
    merged_pokemon.loc[index, 'Attack'] = Attack
    merged_pokemon.loc[index, 'Defense'] = Defense
    merged_pokemon.loc[index, 'SpecialAttack'] = SpecialAttack
    merged_pokemon.loc[index, 'SpecialDefense'] = SpecialDefense
    merged_pokemon.loc[index, 'Speed'] = Speed
    merged_pokemon.loc[index, 'TotalStats'] = TotalStats
    merged_pokemon.loc[index, 'Weight'] = Weight
    merged_pokemon.loc[index, 'Height'] = Height
    merged_pokemon.loc[index, 'GenderProbM'] = GenderProbM
    merged_pokemon.loc[index, 'Category'] = Category
    merged_pokemon.loc[index, 'CatchRate'] = CatchRate
    merged_pokemon.loc[index, 'EggCycles'] = EggCycles
    merged_pokemon.loc[index, 'EggGroup'] = EggGroups
    merged_pokemon.loc[index, 'LevelingRate'] = LevelingRate
    merged_pokemon.loc[index, 'BaseFriendship'] = BaseFriendship
    merged_pokemon.loc[index, 'IsLegendary'] = IsLegendary
    merged_pokemon.loc[index, 'IsMythical'] = IsMythical
    merged_pokemon.loc[index, 'IsUltraBeast'] = IsUltraBeast
    merged_pokemon.loc[index, 'HasMega'] = HasMega
    merged_pokemon.loc[index, 'EvoStage'] = EvoStage
    merged_pokemon.loc[index, 'TotalEvoStages'] = TotalEvoStages
    merged_pokemon.loc[index, 'PreevoName'] = PreevoName
    merged_pokemon.loc[index, 'DamageFromNormal'] = DamageFromNormal
    merged_pokemon.loc[index, 'DamageFromFighting'] = DamageFromFighting
    merged_pokemon.loc[index, 'DamageFromFlying'] = DamageFromFlying
    merged_pokemon.loc[index, 'DamageFromPoison'] = DamageFromPoison
    merged_pokemon.loc[index, 'DamageFromGround'] = DamageFromGround
    merged_pokemon.loc[index, 'DamageFromRock'] = DamageFromRock
    merged_pokemon.loc[index, 'DamageFromBug'] = DamageFromBug
    merged_pokemon.loc[index, 'DamageFromGhost'] = DamageFromGhost
    merged_pokemon.loc[index, 'DamageFromSteel'] = DamageFromSteel
    merged_pokemon.loc[index, 'DamageFromFire'] = DamageFromFire
    merged_pokemon.loc[index, 'DamageFromWater'] = DamageFromWater
    merged_pokemon.loc[index, 'DamageFromGrass'] = DamageFromGrass
    merged_pokemon.loc[index, 'DamageFromElectric'] = DamageFromElectric
    merged_pokemon.loc[index, 'DamageFromPsychic'] = DamageFromPsychic
    merged_pokemon.loc[index, 'DamageFromIce'] = DamageFromIce
    merged_pokemon.loc[index, 'DamageFromDragon'] = DamageFromDragon
    merged_pokemon.loc[index, 'DamageFromDark'] = DamageFromDark
    merged_pokemon.loc[index, 'DamageFromFairy'] = DamageFromFairy

In [907]:
# Mega Venusaur
add_data_mega_primal(
    pokemon_name='Mega Venusaur',
    Abilities='Thick Fat',
    HiddenAbility='[]',
    Generation='I',
    HP=80.0,
    Attack=100.0,
    Defense=123.0,
    SpecialAttack=122.0,
    SpecialDefense=120.0,
    Speed=80.0,
    TotalStats=625.0,
    Weight=155.5,
    Height=2.4,
    GenderProbM='87.5',
    Category='Seed Pokémon',
    CatchRate=45.0,
    EggCycles=20.0,
    EggGroups="['Monster', 'Grass']",
    LevelingRate='Medium Slow',
    BaseFriendship=70.0,
    IsLegendary=0.0,
    IsMythical=0.0,
    EvoStage=3.0,
    TotalEvoStages=3.0,
    PreevoName='Venusaur',
    DamageFromNormal=1.0,
    DamageFromFighting=0.5,
    DamageFromFlying=2.0,
    DamageFromPoison=1.0,
    DamageFromGround=1.0,
    DamageFromRock=1.0,
    DamageFromBug=1.0,
    DamageFromGhost=1.0,
    DamageFromSteel=1.0,
    DamageFromFire=1.0,
    DamageFromWater=0.5,
    DamageFromGrass=0.25,
    DamageFromElectric=0.5,
    DamageFromPsychic=2.0,
    DamageFromIce=1.0,
    DamageFromDragon=1.0,
    DamageFromDark=1.0,
    DamageFromFairy=0.5,
)

# Mega Charizard X
add_data_mega_primal(
    pokemon_name='Mega Charizard X',
    Abilities='Tough Claws',
    HiddenAbility='[]',
    Generation='I',
    HP=78.0,
    Attack=130.0,
    Defense=111.0,
    SpecialAttack=130.0,
    SpecialDefense=85.0,
    Speed=100.0,
    TotalStats=634.0,
    Weight=110.5,
    Height=1.7,
    GenderProbM='87.5',
    Category='Flame Pokémon',
    CatchRate=45.0,
    EggCycles=20.0,
    EggGroups="['Monster', 'Dragon']",
    LevelingRate='Medium Slow',
    BaseFriendship=70.0,
    IsLegendary=0.0,
    IsMythical=0.0,
    EvoStage=3.0,
    TotalEvoStages=3.0,
    PreevoName='Charizard',
    DamageFromNormal=1.0,
    DamageFromFighting=1.0,
    DamageFromFlying=1.0,
    DamageFromPoison=1.0,
    DamageFromGround=2.0,
    DamageFromRock=2.0,
    DamageFromBug=0.5,
    DamageFromGhost=1.0,
    DamageFromSteel=0.5,
    DamageFromFire=0.25,
    DamageFromWater=1.0,
    DamageFromGrass=0.25,
    DamageFromElectric=0.5,
    DamageFromPsychic=1.0,
    DamageFromIce=1.0,
    DamageFromDragon=2.0,
    DamageFromDark=1.0,
    DamageFromFairy=1.0,
)

# Mega Charizard Y
add_data_mega_primal(
    pokemon_name='Mega Charizard Y',
    Abilities='Drought',
    HiddenAbility='[]',
    Generation='I',
    HP=78.0,
    Attack=104.0,
    Defense=78.0,
    SpecialAttack=159.0,
    SpecialDefense=115.0,
    Speed=100.0,
    TotalStats=634.0,
    Weight=100.5,
    Height=1.7,
    GenderProbM='87.5',
    Category='Flame Pokémon',
    CatchRate=45.0,
    EggCycles=20.0,
    EggGroups="['Monster', 'Dragon']",
    LevelingRate='Medium Slow',
    BaseFriendship=70.0,
    IsLegendary=0.0,
    IsMythical=0.0,
    EvoStage=3.0,
    TotalEvoStages=3.0,
    PreevoName='Charizard',
    DamageFromNormal=1.0,
    DamageFromFighting=0.5,
    DamageFromFlying=1.0,
    DamageFromPoison=1.0,
    DamageFromGround=0,
    DamageFromRock=4.0,
    DamageFromBug=0.25,
    DamageFromGhost=1.0,
    DamageFromSteel=0.5,
    DamageFromFire=0.5,
    DamageFromWater=2.0,
    DamageFromGrass=0.25,
    DamageFromElectric=2.0,
    DamageFromPsychic=1.0,
    DamageFromIce=1.0,
    DamageFromDragon=1.0,
    DamageFromDark=1.0,
    DamageFromFairy=0.5,
)

# Mega Blastoise
add_data_mega_primal(
    pokemon_name='Mega Blastoise',
    Abilities='Mega Launcher',
    HiddenAbility='[]',
    Generation='I',
    HP=79.0,
    Attack=103.0,
    Defense=120.0,
    SpecialAttack=135.0,
    SpecialDefense=115.0,
    Speed=78.0,
    TotalStats=630.0,
    Weight=101.1,
    Height=1.6,
    GenderProbM='87.5',
    Category='Shellfish Pokémon',
    CatchRate=45.0,
    EggCycles=20.0,
    EggGroups="['Monster', 'Water 1']",
    LevelingRate='Medium Slow',
    BaseFriendship=70.0,
    IsLegendary=0.0,
    IsMythical=0.0,
    EvoStage=3.0,
    TotalEvoStages=3.0,
    PreevoName='Blastoise',
    DamageFromNormal=1.0,
    DamageFromFighting=1.0,
    DamageFromFlying=1.0,
    DamageFromPoison=1.0,
    DamageFromGround=1.0,
    DamageFromRock=1.0,
    DamageFromBug=1.0,
    DamageFromGhost=1.0,
    DamageFromSteel=0.5,
    DamageFromFire=0.5,
    DamageFromWater=0.5,
    DamageFromGrass=2.0,
    DamageFromElectric=2.0,
    DamageFromPsychic=1.0,
    DamageFromIce=0.5,
    DamageFromDragon=1.0,
    DamageFromDark=1.0,
    DamageFromFairy=1.0,
)

# Mega Beedrill
add_data_mega_primal(
    pokemon_name='Mega Beedrill',
    Abilities='Adaptability',
    HiddenAbility='[]',
    Generation='I',
    HP=65.0,
    Attack=150.0,
    Defense=40.0,
    SpecialAttack=15.0,
    SpecialDefense=80.0,
    Speed=145.0,
    TotalStats=495.0,
    Weight=40.5,
    Height=1.4,
    GenderProbM='50.0',
    Category='Poison Bee Pokémon',
    CatchRate=45.0,
    EggCycles=15.0,
    EggGroups="['Bug']",
    LevelingRate='Medium Fast',
    BaseFriendship=70.0,
    IsLegendary=0.0,
    IsMythical=0.0,
    EvoStage=3.0,
    TotalEvoStages=3.0,
    PreevoName='Beedrill',
    DamageFromNormal=1.0,
    DamageFromFighting=0.25,
    DamageFromFlying=2.0,
    DamageFromPoison=0.5,
    DamageFromGround=1.0,
    DamageFromRock=2.0,
    DamageFromBug=0.5,
    DamageFromGhost=1.0,
    DamageFromSteel=1.0,
    DamageFromFire=2.0,
    DamageFromWater=1.0,
    DamageFromGrass=0.25,
    DamageFromElectric=1.0,
    DamageFromPsychic=2.0,
    DamageFromIce=1.0,
    DamageFromDragon=1.0,
    DamageFromDark=1.0,
    DamageFromFairy=0.5
)

# Mega Pidgeot


<span style = "font-family: Verdana; font-size: 20px">

(Tạm bỏ mấy dòng thiếu để visualize đã, sẽ quay lại fill sau)

In [908]:
merged_pokemon = merged_pokemon.dropna(how='any').reset_index(drop=True)

In [909]:
na = merged_pokemon.isna().sum()
print(f'Missing value in dataset: {sum(na[na > 0])}')

Missing value in dataset: 0


<span style = "font-family: Verdana; font-size: 20px">

##### **3.2. Duplicated Values**
</span>

In [910]:
dup = merged_pokemon.duplicated().sum()
print(f'Duplicated value in dataset: {sum(dup[dup > 0])}')

Duplicated value in dataset: 0


<span style = "font-family: Verdana; font-size: 20px">

##### **3.3. Các vấn đề khác**
</span>

In [911]:
merged_pokemon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 49 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  0 non-null      float64
 1   Name                0 non-null      object 
 2   Type 1              0 non-null      object 
 3   Type 2              0 non-null      object 
 4   Abilities           0 non-null      object 
 5   HiddenAbility       0 non-null      object 
 6   Generation          0 non-null      object 
 7   Hp                  0 non-null      float64
 8   Attack              0 non-null      float64
 9   Defense             0 non-null      float64
 10  SpecialAttack       0 non-null      float64
 11  SpecialDefense      0 non-null      float64
 12  Speed               0 non-null      float64
 13  TotalStats          0 non-null      float64
 14  Weight              0 non-null      float64
 15  Height              0 non-null      float64
 16  GenderProbM         

<span style = "font-family: Verdana; font-size: 20px">

Kiểu dữ liệu của ID đang là int, cần chuyển sang str để tránh nhầm lẫn với các cột số liệu khác sau này
</span>

In [912]:
merged_pokemon['ID'] = merged_pokemon['ID'].astype(str)
merged_pokemon['ID'] = merged_pokemon['ID'].str.replace(r'\.0$', '', regex=True)

<span style = "font-family: Verdana; font-size: 20px">

Kiểu dữ liệu của một số cột cũng nên được chuyển sang int để tránh rắc rối sau này
</span>

In [913]:
for col in ['CatchRate', 'EggCycles', 'BaseFriendship', 'IsLegendary', 'IsMythical', 'IsUltraBeast', 'HasMega', 'EvoStage', 'TotalEvoStages']:
    merged_pokemon[col] = merged_pokemon[col].astype(str)
    merged_pokemon[col] = merged_pokemon[col].str.replace(r'\.0$', '', regex=True)
    merged_pokemon[col] = merged_pokemon[col].astype(int)

<span style = "font-family: Verdana; font-size: 20px">

Các dữ liệu trống của cột `GenderProbM` và `EggGroup` cũng nên được điền lại để thuận tiện cho việc xử lý sau này
</span>

In [914]:
merged_pokemon.loc[merged_pokemon['GenderProbM'] == '-', 'GenderProbM'] = 0
merged_pokemon.loc[merged_pokemon['EggGroup'] == '-', 'EggGroup'] = '[]'

In [915]:
merged_pokemon

,ID,Name,Type 1,Type 2,Abilities,HiddenAbility,Generation,Hp,Attack,Defense,SpecialAttack,SpecialDefense,Speed,TotalStats,Weight,Height,GenderProbM,Category,CatchRate,EggCycles,EggGroup,LevelingRate,BaseFriendship,IsLegendary,IsMythical,IsUltraBeast,HasMega,EvoStage,TotalEvoStages,PreevoName,DamageFromNormal,DamageFromFighting,DamageFromFlying,DamageFromPoison,DamageFromGround,DamageFromRock,DamageFromBug,DamageFromGhost,DamageFromSteel,DamageFromFire,DamageFromWater,DamageFromGrass,DamageFromElectric,DamageFromPsychic,DamageFromIce,DamageFromDragon,DamageFromDark,DamageFromFairy,HP


In [916]:
with open('data/viz_pokemon.csv', 'w', encoding = 'utf-8') as f:
    merged_pokemon.to_csv(f, index=False)